In [1]:
#for disjoint experiment 
def rank_the_disjoint_segments_based_on_predictions(list_of_masks,list_of_segment_avg,pred1,pred2,new_segs):
    a=list_of_masks[pred1]
    b=list_of_masks[pred2]
    
    ranked_a=[]
    ranked_b=[]
    ranked_a_list=[]
    ranked_b_list=[]
    adjacent_segs={}

    aa=list_of_segment_avg[pred1]
    bb=list_of_segment_avg[pred2]
    c={}
    d={}
    for i in range(len(a)):
        c[a[i]]=aa[i]/aa[0]
        d[b[i]]=bb[i]/bb[0]

    # print(c)
    # print(d)
    for i in range(len(a)):
        if c[a[i]]>=d[a[i]]:
            ranked_a.append(a[i])
            ranked_a_list.append({'mask_id':a[i],'required_for_pred':True})
        else:
            ranked_a_list.append({'mask_id':a[i],'required_for_pred':False})
        if d[b[i]] > c[b[i]] :
            ranked_b.append(b[i])
            ranked_b_list.append({'mask_id':b[i],'required_for_pred':True})
        else:
            ranked_b_list.append({'mask_id':b[i],'required_for_pred':False})
        

    # for i in range(len(a)):
    #     if a[i]!=b[i]:
    #         if a[i] not in ranked_b:
    #             ranked_a.append(a[i])
    #             ranked_a_list.append({'mask_id':a[i],'required_for_pred':True})
    #         else:
    #             ranked_a_list.append({'mask_id':a[i],'required_for_pred':False})
    #         if b[i] not in ranked_a:
    #             ranked_b.append(b[i])
    #             ranked_b_list.append({'mask_id':b[i],'required_for_pred':True})
    #         else:
    #             ranked_b_list.append({'mask_id':b[i],'required_for_pred':False})
    #     else:
    #         adjacent_segs[a[i]]=get_adjacent_segments(new_segs,a[i])
    #         #remove these once you figure out how to check for adjacent segments
    #         # ranked_a.append(a[i])
    #         # ranked_b.append(b[i])
    #         ranked_a_list.append({'mask_id':a[i],'required_for_pred':'equal'})
    #         ranked_b_list.append({'mask_id':b[i],'required_for_pred':'equal'})
    
    
    # for i in adjacent_segs.keys():
    #     for j in ranked_a_list:
    #         if i == j['mask_id'] and j['required_for_pred']=='equal':
    #             towards_a=0
    #             towards_b=0
    #             for k in adjacent_segs[i]:
    #                 if k in ranked_a:
    #                     towards_a=towards_a+1
    #                 elif k in ranked_b:
    #                     towards_b=towards_b+1
    #             if towards_a>=towards_b:
    #                 for kk in range(len(ranked_a_list)):
    #                     if ranked_a_list[kk]['mask_id']==i:
    #                         ranked_a_list[kk]['required_for_pred']=True
    #                         ranked_a_list[kk]['once_equals']=True
    #                 for kk in range(len(ranked_b_list)):
    #                     if ranked_b_list[kk]['mask_id']==i:
    #                         ranked_b_list[kk]['required_for_pred']=False
    #                         ranked_b_list[kk]['once_equals']=True
    #             else:
    #                 for kk in range(len(ranked_a_list)):
    #                     if ranked_a_list[kk]['mask_id']==i:
    #                         ranked_a_list[kk]['required_for_pred']=False
    #                         ranked_a_list[kk]['once_equals']=True
    #                 for kk in range(len(ranked_b_list)):
    #                     if ranked_b_list[kk]['mask_id']==i:
    #                         ranked_b_list[kk]['required_for_pred']=True
    #                         ranked_b_list[kk]['once_equals']=True
                        
                    
    # print(adjacent_segs)
    # print('********************************')        
    # print(ranked_a_list)
    # print('********************************')
    # print(ranked_b_list)
            # print(ranked_a_list)
            # print(ranked_b_list)
    return adjacent_segs,ranked_a_list,ranked_b_list


def get_adjacent_segments(new_segs,x):
    shape=new_segs[x].shape
    x1=new_segs[x]
    new_array=copy.deepcopy(x1)
    for i in range(shape[0]):
        for j in range(shape[0]):
            if x1[i][j]==True:
                if i+1 < shape[0] and x1[i+1][j]==False:
                    new_array[i+1][j]=True
                if i-1 >= 0 and x1[i-1][j]==False:
                    new_array[i-1][j]=True
                if j+1 < shape[0] and x1[i][j+1]==False:
                    new_array[i][j+1]=True
                if j-1 >= 0 and x1[i][j-1]==False:
                    new_array[i][j-1]=True
    x3=[]
    for i in range(len(new_segs)):
        x2=np.logical_and(new_segs[i],new_array)
        if np.any(x2)==True and i!=x:
            x3.append(i)
    return x3

    

In [2]:
#for disjoint experiment


def calculate_predictions_for_each_redaction_new(model_in_use,segmentwise_attributions,image,im,topk,top10list,new_segs,prediction1,prediction2,ranked_list,predictions):
    numbered_masks = {ind: mask for ind, mask in enumerate(new_segs)}
    masking_type='normal/'
    dict_for_redacted_images=['temp']
    dict_for_redacted_masks=['temp']
    dict1={}
    dict1[prediction1]=[]
    dict1[prediction2]=[]
    dict1[prediction1].append(predictions[0][prediction1].numpy())
    dict1[prediction2].append(predictions[0][prediction2].numpy())
    dict2={}
    dict2[prediction1]=[]
    dict2[prediction2]=[]
    oldmask=np.zeros(numbered_masks[0].shape, dtype=bool)
    redact_segments_for_this=[True,False]
    
    for k in redact_segments_for_this:
        for i in ranked_list:
            if k == i['required_for_pred']:
                mask=numbered_masks[i['mask_id']]
                mask=np.logical_or(mask,oldmask)
                new_im=mask_on_image(im , mask, masking_type)
                new_im1=mask_on_image(image , mask, masking_type)
                oldmask=mask
                (predictions1,decoded_predictions1,top10list1,top1000list1,id_to_name1)=predict_img(model_in_use,new_im,print_data=False)
                
                # dict_for_redacted_images.append(new_im1) 
                # dict_for_redacted_masks.append(mask) 
                
                dict1[prediction1].append(predictions1[0][prediction1].numpy())
                dict1[prediction2].append(predictions1[0][prediction2].numpy())

        if k == True:
            done_with_one_set=len(dict1[prediction1])
        

    
    
    # print(dict_for_redacted_images)
    for key in dict1:
        for k in range(len(dict1[key])):
            dict2[key].append((dict1[key][k]/dict1[key][0])*100)
    return dict1,dict2,dict_for_redacted_images,dict_for_redacted_masks,done_with_one_set




In [6]:
#for disjoint experiment
def compare_topk(model_in_use,segmentwise_attributions,image,im,topk,top10list,new_segs,predictions,pth,list_of_masks,list_of_segment_avg,redo=False,presoftmax=False):
    if os.path.exists(pth+'_pred_dict'+'.pkl') and redo==False and presoftmax == False:
        with open(pth+'_pred_dict'+'.pkl', 'rb') as file:
            prediction_dictionary=pickle.load(file)
        print('loaded final predictions after redactions from file')
        return prediction_dictionary
    if os.path.exists(pth+'_pred_dict_presoftmax'+'.pkl') and redo==False and presoftmax == True:
        with open(pth+'_pred_dict_presoftmax'+'.pkl', 'rb') as file:
            prediction_dictionary=pickle.load(file)
        print('loaded final predictions_presoftmax after redactions from file')
        return prediction_dictionary
        
    
    
    print('starting redactions')
    combs=[i for i in range(topk)]
    cnt=0
    prediction_dictionary={}
    for (x1,x2) in list(combinations(combs, 2)):
        prediction1=top10list[x1]
        prediction2=top10list[x2]
        prediction_dictionary[f'{prediction1} vs {prediction2}']={}
        prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction1]=[]
        prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction2]=[]
        adjacent_segs,ranked_pred1_list,ranked_pred2_list = rank_the_disjoint_segments_based_on_predictions(list_of_masks,list_of_segment_avg,prediction1,prediction2,new_segs)
        prediction_dictionary1,prediction_dictionary_with_percentages1,dict_for_redacted_images1,dict_for_redacted_masks1,done_with_one_set1=calculate_predictions_for_each_redaction_new(model_in_use,segmentwise_attributions,image,im,topk,top10list,new_segs,prediction1,prediction2,ranked_pred1_list,predictions)
        prediction_dictionary2,prediction_dictionary_with_percentages2,dict_for_redacted_images2,dict_for_redacted_masks2,done_with_one_set2=calculate_predictions_for_each_redaction_new(model_in_use,segmentwise_attributions,image,im,topk,top10list,new_segs,prediction2,prediction1,ranked_pred2_list,predictions)
        # print(dict_for_redacted_images1)
        prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction1].append(prediction_dictionary1)
        prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction1].append(prediction_dictionary_with_percentages1)
        prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction1].append(dict_for_redacted_masks1)
        prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction1].append(done_with_one_set1)
        prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction1].append(ranked_pred1_list)
        
        prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction2].append(prediction_dictionary2)
        prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction2].append(prediction_dictionary_with_percentages2)
        prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction2].append(dict_for_redacted_masks2)
        prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction2].append(done_with_one_set2)
        prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction2].append(ranked_pred2_list)

        prediction_dictionary[f'{prediction1} vs {prediction2}']['adjacent_segs']=adjacent_segs
    print('completed redactions')

    if presoftmax==False:
        with open(pth+'_pred_dict'+'.pkl', 'wb') as file:
            pickle.dump(prediction_dictionary, file)
            file.close()
        return prediction_dictionary
    else:
        with open(pth+'_pred_dict_presoftmax'+'.pkl', 'wb') as file:
            pickle.dump(prediction_dictionary, file)
            file.close()
        return prediction_dictionary
        






In [36]:
#for disjoint experiment

def plot_for_type1_redaction1_new(redacted_image_with_pred1,redacted_mask_with_pred1,first_redaction,id_to_name,prediction1,prediction2,redaction1_percentile,first_redaction_all):
    ROWS=1
    COLS=2
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
    axs.flat[0].imshow(redacted_image_with_pred1)
    axs.flat[1].imshow(redacted_mask_with_pred1)
    
    axs.flat[0].axis('off')
    axs.flat[1].axis('off')
    plt.show()

    ROWS=1
    COLS=2
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-8)))
    axs.flat[0].plot(range(len(first_redaction_all[0][prediction1])),first_redaction_all[0][prediction1],label=id_to_name[prediction1])
    axs.flat[0].plot(range(len(first_redaction_all[0][prediction2])),first_redaction_all[0][prediction2],label=id_to_name[prediction2])
    axs.flat[0].legend()
    axs.flat[0].set_title('probabilities vs no. of redacted segments')
    axs.flat[0].set_ylabel('probabilities')
    axs.flat[0].set_xlabel('no. of redacted segments')
    axs.flat[0].axvline(x=redaction1_percentile, color='r', linestyle=':',label=str(redaction1_percentile))
    axs.flat[0].axvline(x=first_redaction_all[3], color='b', linestyle=':',label=str(first_redaction_all[3]))
    axs.flat[1].plot(range(len(first_redaction[prediction1])),first_redaction[prediction1],label=id_to_name[prediction1])
    axs.flat[1].plot(range(len(first_redaction[prediction2])),first_redaction[prediction2],label=id_to_name[prediction2])
    axs.flat[1].legend()
    # axs.flat[1].set_title('percentages vs no. of redacted segments')
    axs.flat[1].set_ylabel('Percentages',fontsize='12')
    axs.flat[1].set_xlabel('Number of redacted segments',fontsize='12')
    axs.flat[1].axvline(x=redaction1_percentile, color='r', linestyle=':',label=str(redaction1_percentile))
    # axs.flat[1].axvline(x=first_redaction_all[3], color='b', linestyle=':',label=str(first_redaction_all[3]))
    # plt.savefig('zoo/type1_graph1_fig.svg',format='svg')
    plt.show()

def plot_for_type1_redaction2_new(redacted_image_with_pred2,redacted_mask_with_pred2,second_redaction,id_to_name,prediction1,prediction2,redaction2_percentile,second_redaction_all):


    ROWS=1
    COLS=2
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
    axs.flat[0].imshow(redacted_image_with_pred2)
    axs.flat[1].imshow(redacted_mask_with_pred2)
    axs.flat[0].axis('off')
    axs.flat[1].axis('off')
    plt.show()


    ROWS=1
    COLS=2
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-8)))
    axs.flat[0].plot(range(len(second_redaction_all[0][prediction1])),second_redaction_all[0][prediction1],label=id_to_name[prediction1])
    axs.flat[0].plot(range(len(second_redaction_all[0][prediction2])),second_redaction_all[0][prediction2],label=id_to_name[prediction2])
    axs.flat[0].legend()
    axs.flat[0].set_title('probabilities vs no. of redacted segments')
    axs.flat[0].set_ylabel('probabilities')
    axs.flat[0].set_xlabel('no. of redacted segments')
    axs.flat[0].axvline(x=redaction2_percentile, color='r', linestyle=':',label=str(redaction2_percentile))
    axs.flat[0].axvline(x=second_redaction_all[3], color='b', linestyle=':',label=str(second_redaction_all[3]))
    axs.flat[1].plot(range(len(second_redaction[prediction1])),second_redaction[prediction1],label=id_to_name[prediction1])
    axs.flat[1].plot(range(len(second_redaction[prediction2])),second_redaction[prediction2],label=id_to_name[prediction2])
    axs.flat[1].legend()
    # axs.flat[1].set_title('percentages vs no. of redacted segments')
    axs.flat[1].set_ylabel('Percentages',fontsize='12')
    axs.flat[1].set_xlabel('Number of redacted segments',fontsize='12')
    axs.flat[1].axvline(x=redaction2_percentile, color='r', linestyle=':',label=str(redaction2_percentile))
    # axs.flat[1].axvline(x=second_redaction_all[3], color='b', linestyle=':',label=str(second_redaction_all[3]))
    # plt.savefig('zoo/type1_graph2_fig.svg',format='svg')
    plt.show()


def plot_for_type2_sequence_of_masks(segmentwise_attributions,prediction1,prediction2,redaction1_percentile,redaction2_percentile):
    xy=segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100)
    mask_temp=np.zeros(xy.shape)
    mask_temp_array=[]
    mask_temp_array2=[]
    for o in range(redaction1_percentile+1):
        mask_temp_new = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
        if np.all(mask_temp_new== mask_temp):
            continue
        else:
            mask_temp= mask_temp_new
            mask_temp_array.append(mask_temp_new)
            mask_temp_array2.append(o)
   
    n=len(mask_temp_array)                
    # n=redaction1_percentile+1
    ROWS = math.floor(math.sqrt(n))
    COLS = math.ceil(n / ROWS)
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
    for o in range(len(mask_temp_array)):
        axs.flat[o].imshow(mask_temp_array[o])
        axs.flat[o].set_title(mask_temp_array2[o])
    # for o in range(redaction1_percentile+1):
    #     mask_temp = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
    #     axs.flat[o].imshow(mask_temp)
    plt.show()
    
    xy=segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100)
    mask_temp=np.zeros(xy.shape)
    mask_temp_array=[]
    mask_temp_array2=[]
    for o in range(redaction2_percentile+1):
        mask_temp_new = segmentwise_attributions[prediction2] > np.percentile(segmentwise_attributions[prediction2], 100-o)
        if np.all(mask_temp_new== mask_temp):
            continue
        else:
            mask_temp= mask_temp_new
            mask_temp_array.append(mask_temp_new)
            mask_temp_array2.append(o)
    
    
    n=len(mask_temp_array)                
    # n=redaction1_percentile+1
    ROWS = math.floor(math.sqrt(n))
    COLS = math.ceil(n / ROWS)
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
    for o in range(len(mask_temp_array)):
        axs.flat[o].imshow(mask_temp_array[o])
        axs.flat[o].set_title(mask_temp_array2[o])
    # for o in range(redaction1_percentile+1):
    #     mask_temp = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
    #     axs.flat[o].imshow(mask_temp)
    plt.show()

    


In [38]:
#for disjoint experiment



def check_for_type_1_and_2_using_rankings_new(segs,pixelwise_attributions,segmentwise_attributions,prediction_dictionary,model_in_use,im,image,topk,top10list,delta_for_type1,new_segs,list_of_masks,checkonly=None):
    combs=[i for i in range(5)]
    abcd={}
    cnt=0
    for (x1,x2) in list(combinations(combs, 2)):
        prediction1=top10list[x1]
        prediction2=top10list[x2]
        first_redaction=prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction1][1]
        second_redaction=prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction2][1]

        first_redaction_all=prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction1]
        second_redaction_all=prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction2]
        # print(first_redaction_all[2])
        print('checking for combination:',x1,x2)
        print('checking for : ',id_to_name[prediction1],id_to_name[prediction2])
        print('checking for :',prediction1,prediction2)
        type1_flag=None
        type2_flag=None

        print('adjacent segs are ',prediction_dictionary[f'{prediction1} vs {prediction2}']['adjacent_segs'])
        # print()
        
        if checkonly=='type1' or checkonly==None:
            # checking for type1
            print('checking for type1')
            for i in range(len(first_redaction[prediction1])):
                flag1=None
                if first_redaction[prediction1][i] <= 100-delta_for_type1 and first_redaction[prediction2][i] >= delta_for_type1:
                    flag1=True
                    redaction1_percentile=i
                    break
        
            for i in range(len(first_redaction[prediction1])):
                flag2=None    
                if second_redaction[prediction2][i] <= 100-delta_for_type1 and second_redaction[prediction1][i] >= delta_for_type1:
                    flag2=True
                    redaction2_percentile=i
                    break
        
            if flag1==True and flag2==True:
                type1_flag=True
                cnt=cnt+1
                print('Yes its type1')

                
                masking_type='normal/'
                
                print('redaction for '+str(id_to_name[prediction1])+' and no. of redacted segments : ',redaction1_percentile)
                print(' Initial predcition of '+str(id_to_name[prediction1]),first_redaction_all[0][prediction1][0])
                print(' final predcition of '+str(id_to_name[prediction1]),first_redaction_all[0][prediction1][redaction1_percentile])
                print(' final percentage of '+str(id_to_name[prediction1]),first_redaction[prediction1][redaction1_percentile])
                print(' Initial predcition of '+str(id_to_name[prediction2]),first_redaction_all[0][prediction2][0])
                print(' final predcition of '+str(id_to_name[prediction2]),first_redaction_all[0][prediction2][redaction1_percentile])
                print(' final percentage of '+str(id_to_name[prediction2]),first_redaction[prediction2][redaction1_percentile])
    
    
                # redacted_mask_with_pred1=first_redaction_all[2][redaction1_percentile]
                # redacted_mask_with_pred1=get_mask(list_of_masks[prediction1][:redaction1_percentile],new_segs)
                redacted_mask_with_pred1,list_of_segs_for_pred1=get_mask_based_on_disjint_segs_for_red_perc(new_segs,first_redaction_all[4],redaction1_percentile)
                plt.imshow(redacted_mask_with_pred1)
                plt.show()
                redacted_image_with_pred1=mask_on_image(image , redacted_mask_with_pred1, masking_type)
                plot_for_type1_redaction1_new(redacted_image_with_pred1,redacted_mask_with_pred1,first_redaction,id_to_name,prediction1,prediction2,redaction1_percentile,first_redaction_all)
    
                # print(len(first_redaction[prediction1]))
                
                print('redaction for '+str(id_to_name[prediction2])+' and no. of redacted segments : ',redaction2_percentile)
                print(' Initial predcition of '+str(id_to_name[prediction2]),second_redaction_all[0][prediction2][0])
                print(' final predcition of '+str(id_to_name[prediction2]),second_redaction_all[0][prediction2][redaction2_percentile])
                print(' final percentage of '+str(id_to_name[prediction2]),second_redaction[prediction2][redaction2_percentile])
                print(' Initial predcition of '+str(id_to_name[prediction1]),second_redaction_all[0][prediction1][0])
                print(' final predcition of '+str(id_to_name[prediction1]),second_redaction_all[0][prediction1][redaction2_percentile])
                print(' final percentage of '+str(id_to_name[prediction1]),second_redaction[prediction1][redaction2_percentile])
    

                # redacted_mask_with_pred2=second_redaction_all[2][redaction2_percentile]
                # redacted_mask_with_pred2=get_mask(list_of_masks[prediction2][:redaction2_percentile],new_segs)
                redacted_mask_with_pred2,list_of_segs_for_pred2=get_mask_based_on_disjint_segs_for_red_perc(new_segs,second_redaction_all[4],redaction2_percentile)
                redacted_image_with_pred2=mask_on_image(image , redacted_mask_with_pred2, masking_type)
                plot_for_type1_redaction2_new(redacted_image_with_pred2,redacted_mask_with_pred2,second_redaction,id_to_name,prediction1,prediction2,redaction2_percentile,second_redaction_all)
    
                
                # mask_for_redaction1 = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-redaction1_percentile)
                # mask_for_redaction2 = segmentwise_attributions[prediction2] > np.percentile(segmentwise_attributions[prediction2], 100-redaction2_percentile)
        
                
                
                # print(id_to_name[prediction1])
                # plt.imshow(redacted_image_with_pred1)
                # plt.show()
                # print(id_to_name[prediction2])
                # plt.imshow(redacted_image_with_pred2)
                # plt.show()
        # try:
        #     abcd[f'{prediction1} vs {prediction2}']=[redacted_mask_with_pred1,redacted_mask_with_pred2,list_of_segs_for_pred1,list_of_segs_for_pred2]
        #     # if x1==0 and x2==1:
        #     #     break
        # except:
            abcd[f'{prediction1} vs {prediction2}']=[]
        '''if checkonly=='type2' or checkonly==None and type1_flag!=True:
            #checking for type2
            print('checking for type2')
            # print(first_redaction[prediction1])
            # print(first_redaction[prediction2])
            for i in range(len(first_redaction[prediction1])):
                flag1=None
                # print(i,first_redaction[prediction1][i],first_redaction[prediction2][i])
                if first_redaction[prediction1][i] <= 20 and first_redaction[prediction2][i] <= 20:
                    flag1=True
                    redaction1_percentile=i
                    break
        
            for i in range(len(first_redaction[prediction1])):
                flag2=None
                # print(i,second_redaction[prediction2][i],second_redaction[prediction1][i])
        
                if second_redaction[prediction2][i] <= 20 and second_redaction[prediction1][i] <= 20:
                    flag2=True
                    redaction2_percentile=i
                    break
            # print(flag1,flag2)
            if flag1==True and flag2==True:
                
                # mask_for_redaction1 = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-redaction1_percentile)
                # mask_for_redaction2 = segmentwise_attributions[prediction2] > np.percentile(segmentwise_attributions[prediction2], 100-redaction2_percentile)
    
                # plt.imshow(mask_for_redaction1)
                # plt.show()
                # plt.imshow(new_im1)
                # plt.show()
                # plt.imshow(mask_for_redaction2)
                # plt.show()
                # plt.imshow(new_im1)
                # plt.show()
                # mask_union=np.logical_or(mask_for_redaction1, mask_for_redaction2) 
                # mask_intersection=np.logical_and(mask_for_redaction1, mask_for_redaction2) 
                # mask_only_for_1=(mask_union.astype(float)-mask_for_redaction2.astype(float)).astype(bool)
                # mask_only_for_2=(mask_union.astype(float)-mask_for_redaction1.astype(float)).astype(bool)
                
                
        
                # redacted_image_with_pred1,redacted_mask_with_pred1=show_redacted_image(segmentwise_attributions,prediction1,redaction1_percentile)
                # redacted_image_with_pred2,redacted_mask_with_pred2=show_redacted_image(segmentwise_attributions,prediction2,redaction2_percentile)
                
                
                # masking_type='normal/'
                # new_im=mask_on_image(im , mask_only_for_1, masking_type)
                # new_im1=mask_on_image(image , mask_only_for_1, masking_type)
                # (predictions1,decoded_predictions1,top10list1,top1000list1,id_to_name1)=predict_img(model_in_use,new_im,print_data=False)
                # new_prediction1_percentage_after_only1=(predictions1[0][prediction1].numpy()/final_prediction_dictionary[prediction1][prediction1][0])*100
                # new_prediction2_percentage_after_only1=(predictions1[0][prediction2].numpy()/final_prediction_dictionary[prediction2][prediction2][0])*100
                
                # # plt.imshow(mask_only_for_1)
                # # plt.show()
                # # plt.imshow(new_im1)
                # # plt.show()
                # masking_type='normal/'
                # new_im=mask_on_image(im , mask_only_for_2, masking_type)
                # new_im1=mask_on_image(image , mask_only_for_2, masking_type)
                # (predictions2,decoded_predictions2,top10list2,top1000list2,id_to_name2)=predict_img(model_in_use,new_im,print_data=False)
                # new_prediction1_percentage_after_only2=(predictions2[0][prediction1].numpy()/final_prediction_dictionary[prediction1][prediction1][0])*100
                # new_prediction2_percentage_after_only2=(predictions2[0][prediction2].numpy()/final_prediction_dictionary[prediction2][prediction2][0])*100
                # # plt.imshow(mask_only_for_2)
                # # plt.show()
                
                # masking_type='normal/'
                # new_im=mask_on_image(im , mask_intersection, masking_type)
                # new_im1=mask_on_image(image , mask_intersection, masking_type)
                # (predictions3,decoded_predictions3,top10list3,top1000list3,id_to_name3)=predict_img(model_in_use,new_im,print_data=False)
                # new_prediction1_percentage_after_intersection=(predictions3[0][prediction1].numpy()/final_prediction_dictionary[prediction1][prediction1][0])*100
                # new_prediction2_percentage_after_intersection=(predictions3[0][prediction2].numpy()/final_prediction_dictionary[prediction2][prediction2][0])*100
                
               
                
                
                
                masking_type='normal/'
                
                print('redaction for '+str(prediction1)+' percentile at ',redaction1_percentile)
                print(' Initial predcition of '+str(prediction1),first_redaction_all[0][prediction1][0])
                print(' final predcition of '+str(prediction1),first_redaction_all[0][prediction1][redaction1_percentile])
                print(' final percentage of '+str(prediction1),first_redaction[prediction1][redaction1_percentile])
                print(' Initial predcition of '+str(prediction2),first_redaction_all[0][prediction2][0])
                print(' final predcition of '+str(prediction2),first_redaction_all[0][prediction2][redaction1_percentile])
                print(' final percentage of '+str(prediction2),first_redaction[prediction2][redaction1_percentile])
    
    
                redacted_mask_with_pred1=first_redaction_all[2][redaction1_percentile]
                redacted_image_with_pred1=mask_on_image(image , redacted_mask_with_pred1, masking_type)
                plot_for_type1_redaction1_new(redacted_image_with_pred1,redacted_mask_with_pred1,first_redaction,id_to_name,prediction1,prediction2,redaction1_percentile,first_redaction_all)
    
                # print(len(first_redaction[prediction1]))
                
                print('redaction for '+str(prediction2)+' percentile at ',redaction2_percentile)
                print(' Initial predcition of '+str(prediction2),second_redaction_all[0][prediction2][0])
                print(' final predcition of '+str(prediction2),second_redaction_all[0][prediction2][redaction2_percentile])
                print(' final percentage of '+str(prediction2),second_redaction[prediction2][redaction2_percentile])
                print(' Initial predcition of '+str(prediction1),second_redaction_all[0][prediction1][0])
                print(' final predcition of '+str(prediction1),second_redaction_all[0][prediction1][redaction2_percentile])
                print(' final percentage of '+str(prediction1),second_redaction[prediction1][redaction2_percentile])
    

                redacted_mask_with_pred2=second_redaction_all[2][redaction2_percentile]
                redacted_image_with_pred2=mask_on_image(image , redacted_mask_with_pred2, masking_type)
                plot_for_type1_redaction2_new(redacted_image_with_pred2,redacted_mask_with_pred2,second_redaction,id_to_name,prediction1,prediction2,redaction2_percentile,second_redaction_all)
    

                
                
                # prediction1=top10list[1]
                # prediction2=top10list[2]
                # prediction_dictionary[f'{prediction1} vs {prediction2}']['adjacent_segs']
                mask_temp_array=prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction1][2][1:]
                n=len(mask_temp_array)                
                # n=redaction1_percentile+1
                ROWS = math.floor(math.sqrt(n))
                COLS = math.ceil(n / ROWS)
                UPSCALE_FACTOR = 10
                fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
                for o in range(len(mask_temp_array)):
                    axs.flat[o].imshow(mask_temp_array[o])
                    # axs.flat[o].set_title(mask_temp_array2[o])
                # for o in range(redaction1_percentile+1):
                #     mask_temp = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
                #     axs.flat[o].imshow(mask_temp)
                plt.show()



                print('******************************************')
                # prediction1=top10list[1]
                # prediction2=top10list[2]
                # prediction_dictionary[f'{prediction1} vs {prediction2}']['adjacent_segs']
                temp_array=list_of_masks[prediction1]
                # mask_temp_array=prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction1][2][1:]
                n=len(temp_array)                
                # n=redaction1_percentile+1
                ROWS = math.floor(math.sqrt(n))
                COLS = math.ceil(n / ROWS)
                UPSCALE_FACTOR = 10
                fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
                for o in range(len(temp_array)):
                    axs.flat[o].imshow(new_segs[temp_array[o]])
                    axs.flat[o].set_title(f'{temp_array[o]}')
                # for o in range(redaction1_percentile+1):
                #     mask_temp = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
                #     axs.flat[o].imshow(mask_temp)
                plt.show()
                
                print('******************************************')
                
                mask_temp_array=prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction2][2][1:]
                n=len(mask_temp_array)                
                # n=redaction1_percentile+1
                ROWS = math.floor(math.sqrt(n))
                COLS = math.ceil(n / ROWS)
                UPSCALE_FACTOR = 10
                fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
                for o in range(len(mask_temp_array)):
                    axs.flat[o].imshow(mask_temp_array[o])
                    # axs.flat[o].set_title(mask_temp_array2[o])
                # for o in range(redaction1_percentile+1):
                #     mask_temp = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
                #     axs.flat[o].imshow(mask_temp)
                plt.show()
                


                print('******************************************')
                
                # prediction1=top10list[1]
                # prediction2=top10list[2]
                # prediction_dictionary[f'{prediction1} vs {prediction2}']['adjacent_segs']
                temp_array=list_of_masks[prediction2]
                # mask_temp_array=prediction_dictionary[f'{prediction1} vs {prediction2}'][prediction1][2][1:]
                n=len(temp_array)                
                # n=redaction1_percentile+1
                ROWS = math.floor(math.sqrt(n))
                COLS = math.ceil(n / ROWS)
                UPSCALE_FACTOR = 10
                fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
                for o in range(len(temp_array)):
                    axs.flat[o].imshow(new_segs[temp_array[o]])
                    axs.flat[o].set_title(f'{temp_array[o]}')
                # for o in range(redaction1_percentile+1):
                #     mask_temp = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
                #     axs.flat[o].imshow(mask_temp)
                plt.show()    
                                    


    
                # plt.imshow(mask_only_for_2)
                # plt.show()
                # plt.imshow(new_im1)
                # plt.show()
                # print(id_to_name[prediction1])
                # plt.imshow(redacted_image_with_pred1)
                # plt.show()
                # plt.imshow(mask_for_redaction1)
                # plt.show()
                # # plt.imshow(mask_only_for_1)
                # # plt.show()
                # print(id_to_name[prediction2])
                # plt.imshow(redacted_image_with_pred2)
                # plt.show()
                # plt.imshow(mask_for_redaction2)
                # plt.show()
                # # plt.imshow(mask_only_for_2)
                # # plt.show()
                # plt.imshow(mask_union)
                # plt.show()
                # plt.imshow(mask_intersection)
                # plt.show()'''
        # break
    # if type1_flag==True and type2_flag==True:
    #     print('Here i am , Both are true')
    # print(cnt)
    
        
    return abcd,cnt
    
    
    


